In [1]:
import requests
import pandas as pd
from anytree import Node, RenderTree
import random
import string
from functions import *

In [2]:
with open('Authentication/database_uri.txt', 'r', encoding="utf8") as f:
    uri = f.read()

In [3]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, next_token = None):    
    response = requests.request("GET", url, headers = headers)
        
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def getTweets(user_id, header):
    tweets_url = f'https://api.twitter.com/2/users/{user_id}/tweets'
    return connect_to_endpoint(tweets_url, header)

# 'conversation_id' is the identifier for the main tweet
def getConversation(conversation_id, max_results, header):
    params = 'in_reply_to_user_id,author_id,created_at,conversation_id'
    getConversation_url = f'https://api.twitter.com/2/tweets/search/recent?query=conversation_id:{conversation_id}&tweet.fields={params}&max_results={max_results}'

    return connect_to_endpoint(getConversation_url, header)

# For now we will return the time only
def getTweetInformation(conversation_id, header):
    params = 'created_at,conversation_id,in_reply_to_user_id,author_id,referenced_tweets'
    tweetInfo_url = f'https://api.twitter.com/2/tweets?tweet.fields={params}&ids={conversation_id}'
    
    result = connect_to_endpoint(tweetInfo_url, header)
    return result['data'][0]['created_at']

def getTweetComments(conversation_data):
    conversation_dict = {'id':[], 'timestamp':[], 'reply_to':[], 'comment':[], 'new_id':[]}
    
    for i in range(len(conversation_data['data'])):
        print('User ID:', conversation_data['data'][i]['author_id'], 
              'Time:', conversation_data['data'][i]['created_at'])
        print('In reply to:', conversation_data['data'][i]['in_reply_to_user_id'])
        print(conversation_data['data'][i]['text'], '\n')
        
        conversation_dict['id'].append(conversation_data['data'][i]['author_id'])
        conversation_dict['timestamp'].append(conversation_data['data'][i]['created_at'])
        conversation_dict['reply_to'].append(conversation_data['data'][i]['in_reply_to_user_id'])
        conversation_dict['comment'].append(conversation_data['data'][i]['text'])
        conversation_dict['new_id'].append(generateRandomID())
        
    return conversation_dict

def generateRandomID(strlen = 8):
    return ''.join(random.choice(string.ascii_uppercase + string.ascii_lowercase + string.digits) for _ in range(strlen))

def getParentID(dataframe):
    parent_id = []
    root = generateRandomID()

    for index, row in df.iterrows():
        if row['reply_to'] != '87818409':
            count = 0
            
            for _, i in df.iterrows():
                if row['reply_to'] == i['id']:
                    parent_id.append(i['new_id'])
                    break
                
                count += 1

                if count == len(df) + 1:
                    print('lol')
                    parent_id.append(i['nan'])
        
        else:
            parent_id.append(root)
            
    return parent_id

In [4]:
with open('Authentication/twitter_bearer_token.txt', 'r', encoding="utf8") as f:
    token = f.read()

header = create_headers(token)
max_results = 100

# The Straits Times
## Get Tweets

In [5]:
ST_id = '37874853'
ST_tweets = getTweets(ST_id, header)

In [6]:
for i in range(len(ST_tweets['data'])):
    print('Tweet ID:', ST_tweets['data'][i]['id'], 
          'Time:', getTweetInformation(ST_tweets['data'][i]['id'], header))
    print(ST_tweets['data'][i]['text'], '\n')
    
#     command = (
#             '''
#             INSERT INTO twitter_data
#             VALUES ('%s', '%s', '%s');
#             ''' % (ST_tweets['data'][i]['id'], getTweetInformation(ST_tweets['data'][i]['id'], header), 
#                    ST_tweets['data'][i]['text'])
#             )
#     setUpDB(command, uri)

Tweet ID: 1491924206153392129 Time: 2022-02-10T23:57:04.000Z
RT @stbusinessdesk: International bookings to South-east Asia rising as borders open: Skyscanner https://t.co/1pNKAIFCZE 

Tweet ID: 1491924201023754243 Time: 2022-02-10T23:57:02.000Z
RT @stbusinessdesk: Global patent filings surged to record high in 2021, up 23% from Singapore: UN report https://t.co/z0pQNkATRe 

Tweet ID: 1491918813352185859 Time: 2022-02-10T23:35:38.000Z
RT @STForeignDesk: US lawmakers introduce bipartisan Bill to address social media addiction https://t.co/tk8E0fXMtI 

Tweet ID: 1491918806746230786 Time: 2022-02-10T23:35:36.000Z
RT @STForeignDesk: Masks in class: How damaging to child development? https://t.co/xPB5vdaKbh 

Tweet ID: 1491918799934611456 Time: 2022-02-10T23:35:35.000Z
RT @STForeignDesk: Canada seeks to join EU challenge against China at WTO https://t.co/ZO4jJxjA92 

Tweet ID: 1491918792783323136 Time: 2022-02-10T23:35:33.000Z
RT @STForeignDesk: While You Were Sleeping: 5 stories you might h

In [7]:
# query = '@straits_times'
# tweets_url = f'https://api.twitter.com/2/tweets/search/recent?query={query}&max_results={max_results}'
# connect_to_endpoint(tweets_url, header)

## Get Conversation

In [8]:
# ST_conversation_data = getConversation('1475383966832021504', max_results, header)
ST_conversation_data = getConversation('1389937492065931266', max_results, header)

In [9]:
ST_result = getTweetComments(ST_conversation_data)

User ID: 1435908999572955139 Time: 2022-02-10T22:30:38.000Z
In reply to: 87818409
@guardian You say your independent with ‘values’ not sure who’s values they are, but when you headline an article with ‘antivax’s’ when only a small portion of those people are antivax and you stereotype everyone else in that demonstration whom may be pro vax and fully vax, then you fail! 

User ID: 1394744372097339393 Time: 2022-02-10T19:45:16.000Z
In reply to: 87818409
@guardian Times certainly do change, any thoughts on this...
https://t.co/dtufbIGrv5 

User ID: 1267097982446469123 Time: 2022-02-10T18:49:35.000Z
In reply to: 87818409
@guardian sick English fascist nation, are you asking Russia to withdraw its army from its territory ?! And you criminals to continue killing peoples in other countries ?! 

User ID: 884725084371521536 Time: 2022-02-10T15:17:25.000Z
In reply to: 87818409
@guardian Values like client journalism and vendetta against a mild democratic socialist? Still lying in your paper.
htt

In [10]:
ST_df = pd.DataFrame.from_dict(ST_result)
ST_df['id'] = ST_df['id'].astype(str)
ST_df['reply_to'] = ST_df['reply_to'].astype(str)
ST_df['url'] = ST_df['comment'].apply(lambda x: getLinks(x))
ST_df['link_title'] = ST_df['url'].apply(lambda x: getURLfromList(x))
# ST_df.to_csv('Datasets/ST_twitter_data.csv', index=False)

ST_df.head()

,id,timestamp,reply_to,comment,new_id,url,link_title
0,1435908999572955139,2022-02-10T22:30:38.000Z,87818409,@guardian You say your independent with ‘value...,2QemUVDg,,
1,1394744372097339393,2022-02-10T19:45:16.000Z,87818409,"@guardian Times certainly do change, any thoug...",E0cpM7If,https://t.co/dtufbIGrv5;,[t.co]
2,1267097982446469123,2022-02-10T18:49:35.000Z,87818409,"@guardian sick English fascist nation, are you...",sr4Xz6XE,,
3,884725084371521536,2022-02-10T15:17:25.000Z,87818409,@guardian Values like client journalism and ve...,u4V0tsUf,https://twitter.com/NicCicutti/status/14917010...,[twitter.com]
4,1001839970221469696,2022-02-10T14:25:51.000Z,87818409,@guardian https://t.co/q58wQRExSe,ODG0l3kr,https://twitter.com/Linn45350887/status/149177...,[twitter.com]


# Channel NewsAsia
## Get Tweets

In [11]:
CNA_tweets = getTweets('38400130', header)

for i in range(len(CNA_tweets['data'])):
    print('Tweet ID:', CNA_tweets['data'][i]['id'],
          'Time:', getTweetInformation(ST_tweets['data'][i]['id'], header))
    print(CNA_tweets['data'][i]['text'], '\n')
    
#     command = (
#             '''
#             INSERT INTO twitter_data
#             VALUES ('%s', '%s', '%s');
#             ''' % (CNA_tweets['data'][i]['id'], getTweetInformation(CNA_tweets['data'][i]['id'], header), 
#                    CNA_tweets['data'][i]['text'])
#             )
#     setUpDB(command, uri)

Tweet ID: 1491923704913166337 Time: 2022-02-10T23:57:04.000Z
International bookings to Southeast Asia rising as borders open: Skyscanner https://t.co/gq5CZmq7Y5 https://t.co/2O30DV2Tci 

Tweet ID: 1491921942772785152 Time: 2022-02-10T23:57:02.000Z
International bookings to Southeast Asia rising as borders open -Skyscanner https://t.co/PprkuOhvV2 https://t.co/ZPKCTzHGuf 

Tweet ID: 1491916667068157957 Time: 2022-02-10T23:35:38.000Z
'He already looks like a Liverpool player'- Klopp praises debutant Diaz https://t.co/HRZz6O2LxZ https://t.co/fJhnZJ20Zu 

Tweet ID: 1491916658172067844 Time: 2022-02-10T23:35:36.000Z
A Valentine's Day staycation with a DIY meal kit... and other ways to indulge with minimal effort https://t.co/DypGrrHbat https://t.co/TkkfmmkhvZ 

Tweet ID: 1491914895666737155 Time: 2022-02-10T23:35:35.000Z
US lawmakers introduce bipartisan bill to address social media addiction https://t.co/e8hFhqcJ0k https://t.co/02hd2swkFY 

Tweet ID: 1491913137628463105 Time: 2022-02-10T23:

## Get Conversation

In [12]:
CNA_conversation_data = getConversation('1389937492065931266', max_results, header)
# CNA_conversation_data = getConversation('1477849910682832897', max_results, header)

result = getTweetComments(CNA_conversation_data)

User ID: 1435908999572955139 Time: 2022-02-10T22:30:38.000Z
In reply to: 87818409
@guardian You say your independent with ‘values’ not sure who’s values they are, but when you headline an article with ‘antivax’s’ when only a small portion of those people are antivax and you stereotype everyone else in that demonstration whom may be pro vax and fully vax, then you fail! 

User ID: 1394744372097339393 Time: 2022-02-10T19:45:16.000Z
In reply to: 87818409
@guardian Times certainly do change, any thoughts on this...
https://t.co/dtufbIGrv5 

User ID: 1267097982446469123 Time: 2022-02-10T18:49:35.000Z
In reply to: 87818409
@guardian sick English fascist nation, are you asking Russia to withdraw its army from its territory ?! And you criminals to continue killing peoples in other countries ?! 

User ID: 884725084371521536 Time: 2022-02-10T15:17:25.000Z
In reply to: 87818409
@guardian Values like client journalism and vendetta against a mild democratic socialist? Still lying in your paper.
htt

In [13]:
df = pd.DataFrame.from_dict(result)
df['id'] = df['id'].astype(str)
df['reply_to'] = df['reply_to'].astype(str)
df['url'] = df['comment'].apply(lambda x: getLinks(x))
df['link_title'] = df['url'].apply(lambda x: getURLfromList(x))
# df = df.drop([14, 17])
df['parent_id'] = getParentID(df)
df.to_csv('Datasets/sample_tweet_conversation.csv', index=False)

df.head()

,id,timestamp,reply_to,comment,new_id,url,link_title,parent_id
0,1435908999572955139,2022-02-10T22:30:38.000Z,87818409,@guardian You say your independent with ‘value...,KbLfC5GE,,,24JkpHAK
1,1394744372097339393,2022-02-10T19:45:16.000Z,87818409,"@guardian Times certainly do change, any thoug...",UetMh5h7,https://t.co/dtufbIGrv5;,[t.co],24JkpHAK
2,1267097982446469123,2022-02-10T18:49:35.000Z,87818409,"@guardian sick English fascist nation, are you...",cDYrS6fv,,,24JkpHAK
3,884725084371521536,2022-02-10T15:17:25.000Z,87818409,@guardian Values like client journalism and ve...,8jrlTr58,https://twitter.com/NicCicutti/status/14917010...,[twitter.com],24JkpHAK
4,1001839970221469696,2022-02-10T14:25:51.000Z,87818409,@guardian https://t.co/q58wQRExSe,mMopkhLx,https://twitter.com/Linn45350887/status/149177...,[twitter.com],24JkpHAK
